In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "2024-09-25"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

instrument = "LATISS"
observation_reason = "ALL"
observation_type = "science"
science_program = "ALL"

In [2]:
import os
import datetime as dt
import pandas as pd
from IPython.display import HTML, display

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
import lsst.ts.logging_and_reporting.source_adapters as sad
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md

# Set default env to "usdf" and try before PUSH to repo.

# The default provided here is for local testing.
# Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", ut.Server.summit
)  # TODO try with "usdf" before push (else "summit")

# Normalize Parameters (both explicit Times Squares params and implicit ones)
observation_reason = None if observation_reason=='ALL' else observation_reason
observation_type = None if observation_type=='ALL' else observation_type
science_program = None if science_program=='ALL' else science_program
params = dict(
    day_obs=day_obs,
    number_of_days=number_of_days,
    instrument=instrument,
    observation_reason=observation_reason,
    observation_type=observation_type,
    science_program=science_program,
)


In [3]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

In [4]:
src_exp = sad.ExposurelogAdapter(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    limit=5000,
)

In [5]:
df = src_exp.exposure_detail(instrument=instrument,
                            science_program=science_program,
                            observation_reason=observation_reason,
                            observation_type=observation_type
                           )
md(f'The number of exposures in this filtered result is {len(df.index)}')
display(HTML(df.to_html()))

The number of exposures in this filtered result is 166

,obs id,timespan begin,seq num,observation type,observation reason,science program,exposure time
0,AT_O_20240925_000252,2024-09-26T04:35:31.405002,252,science,x_offset_-50,spec-survey,0.0
1,AT_O_20240925_000251,2024-09-26T04:34:05.867988,251,science,x_offset_0,spec-survey,0.0
2,AT_O_20240925_000250,2024-09-26T04:32:42.858984,250,science,object,spec-survey,0.0
3,AT_O_20240925_000249,2024-09-26T04:31:29.853989,249,science,object,spec-survey,0.0
4,AT_O_20240925_000246,2024-09-26T04:28:20.368997,246,science,imaging,BLOCK-306,0.0
5,AT_O_20240925_000245,2024-09-26T04:27:33.485003,245,science,imaging,BLOCK-306,0.0
6,AT_O_20240925_000244,2024-09-26T04:26:04.314012,244,science,imaging,BLOCK-306,0.0
7,AT_O_20240925_000243,2024-09-26T04:25:17.402016,243,science,imaging,BLOCK-306,0.0
8,AT_O_20240925_000242,2024-09-26T04:23:46.893997,242,science,imaging,BLOCK-306,0.0
9,AT_O_20240925_000241,2024-09-26T04:23:00.055989,241,science,imaging,BLOCK-306,0.0


-----------------

# Developer Only Section

In [6]:
display(params)  
print(src_exp)
print(f'{instrument=}, {science_program=}, {observation_reason=}, {observation_type=}')

{'day_obs': '2024-09-25',
 'number_of_days': '1',
 'instrument': 'LATISS',
 'observation_reason': None,
 'observation_type': 'science',
 'science_program': None}

https://summit-lsp.lsst.codes: 2024-09-25 to 2024-09-26 [5000] exposurelog endpoints=['instruments', 'exposures', 'messages']
instrument='LATISS', science_program=None, observation_reason=None, observation_type='science'


In [7]:
print({k: len(v) for k,v in src_exp.exposures.items()})

{'LSSTComCamSim': 0, 'LSSTComCam': 0, 'LATISS': 252}


In [8]:
src_exp.exposure_detail(instrument=instrument,
                            science_program=science_program,
                            observation_reason=observation_reason,
                            observation_type=observation_type
                           )


,obs id,timespan begin,seq num,observation type,observation reason,science program,exposure time
0,AT_O_20240925_000252,2024-09-26T04:35:31.405002,252,science,x_offset_-50,spec-survey,0.0
1,AT_O_20240925_000251,2024-09-26T04:34:05.867988,251,science,x_offset_0,spec-survey,0.0
2,AT_O_20240925_000250,2024-09-26T04:32:42.858984,250,science,object,spec-survey,0.0
3,AT_O_20240925_000249,2024-09-26T04:31:29.853989,249,science,object,spec-survey,0.0
4,AT_O_20240925_000246,2024-09-26T04:28:20.368997,246,science,imaging,BLOCK-306,0.0
...,...,...,...,...,...,...,...
161,AT_O_20240925_000025,2024-09-25T23:49:42.340989,25,science,x_offset_50,spec-survey,0.0
162,AT_O_20240925_000024,2024-09-25T23:48:15.023015,24,science,x_offset_-50,spec-survey,0.0
163,AT_O_20240925_000023,2024-09-25T23:46:51.609989,23,science,x_offset_0,spec-survey,0.0
164,AT_O_20240925_000022,2024-09-25T23:45:29.069017,22,science,object,spec-survey,0.0


In [9]:
#src_exp.exposures['LATISS']